# Chatbot infos corona

Dans ce cas pratique on va essayer d'entraîner un chatbot qui permette de répondre automatiquement aux questions de l'utilisateur sur le coronavirus à partir de multiples informations que l'on a regroupées dans le fichier `infos_corona.txt`. L'idée est que le chatbot renvoie la ou les phrases utilisant le plus de termes similaires à ceux utilisés dans la question de l'utilisateur.   
La démarche est similaire à celle présentée dans le notebook `chatbot_wiki`.  

- En utilisant la fonction `open` (et en précisant le paramètre `encoding` adéquat!), importez le texte dans une chaîne de caractères que vous nommerez `texte`.  
- Procédez à quelques nettoyages : 
    - passez le texte en minuscules.  
    - vérifiez s'il n'y pas d'acronyme pouvant fausser notre tokenization en phrases, si oui, remplacez-le par autre chose.  
    - le virus est appelé covid-19 ou coronavirus, faites en sorte qu'un terme unique soit utilisé.   
- Avec la fonction `nltk.sent_tokenize`, passez votre chaîne de caractères `texte` en une liste de phrases que vous appelerez `phrases_token`.  
- Il y a beaucoup de questions dans cette liste, or on veut des réponses. Supprimez-les.
- Récupérez un vecteur de stop words français avec la fonction `get_stop_words` du module `stop_words` 

On a maintenant tout ce dont on a besoin pour faire notre matrice TF-IDF! Vous pouvez déjà la fitter sur vos infos :  
- Stockez le résultat de `TfidfVectorizer` en fixant le paramètre `stop_words` avec les stop words français.  
- Fittez la fonction sur votre liste de phrases `phrases_token` et stockez ce résultat dans un objet `tf_idf_chat`.  

Il faut maintenant définir une fonction que vous appelerez dans votre chatbot. Celle-ci doit :  
- Prendre en entrée la phrase entrée par l'utilisateur et la mettre dans une liste.    
- Créer la matrice TF-IDF des infos avec `tf_idf_chat.transform()`. 
- Créer la matrice TF-IDF pour la phrase de l'utilisateur avec `tf_idf_chat.transform()`.  
- Calculez la similarité entre la phrase de l'utilisateur et le reste des phrases avec `sklearn.metrics.pairwise.cosine_similarity`.  
- Renvoyer en réponse la phrase avec la similarité la plus grande, ou un message d'erreur s'il n'y a pas de similarité. _Alternative un peu plus complexe_ : Vous pouvez aussi renvoyer plusieurs phrases si plusieurs sont similaires, en les concaténant dans une même chaîne de caractère avec '\n'.join().  

Maintenant il n'y a plus qu'à intégrer cette fonction dans votre chatbot et le tester. N'oubliez pas de lui faire dire bonjour et de laisser la possibilité à l'utilisateur de quitter!

## Rendez votre chatbot encore plus intelligent  

Donnez la possibilité à l'utilisateur de nourrir le chatbot de nouvelles informations. Vous pouvez par exemple déterminer qu'après avoir tapé "infos", l'utilisateur va rentrer une phrase que vous devrez ajouter aux possibilités de réponses de votre chatbot.  

## Bonus avec stemmatizer

On définit maintenant une fonction à indiquer dans le paramètre `tokenizer` de la fonction `TfidfVectorizer`. Cette fonction doit récupérer la racine des mots plutôt que les mots en entier pour trouver des correspondances entre des mots de la même racine même s'ils ne sont pas écrits sous la même forme.  
- Utilisez la fonction `FrenchStemmer` de `nltk.stem.snowball` pour définir :  
    - une première fonction qui renvoie la liste des mots stemmatisés quand on rentre une liste de mots.  
    - Une fonction qui applique cette première fonction à une phrase qu'on tokenize en mots (avec `nltk.word_tokenize`)  
Vous pouvez tester cette fonction sur des phrases pour voir si elle fait bien ce que vous désirez.  
- Ajoutez l'appel à cette fonction dans `TfidfVectorizer`.  

In [1]:
import nltk
import re
import string
import numpy as np
import random

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

with open('infos_corona.txt','r',errors = 'ignore', encoding = "utf8") as f:
    texte = f.read()

#nltk.download('punkt')
#nltk.download('wordnet')

# on supprime l'acronyme n.c.a, on corrige : 
texte = re.sub('n.c.a.', 'nca', texte)

# Tokenisation en phrases et mots
phrases_token = nltk.sent_tokenize(texte, language = "french")

# on a beaucoup de questions ici : on les enlève
for i in sorted(range(len(phrases_token)), reverse = True):
    if re.search(r"\?", phrases_token[i]):
        del phrases_token[i]

# on enlève les doublons
phrases_token = list(set(phrases_token)) 

# On crée une liste nettoyée mais qui ne sera pas celle dans laquelle
# on ira chercher les réponses, simplement pour la création de la
# matrice TF-IDF
def nettoyage(texte):
    texte = texte.lower()
    # on remplace covid-19 par coronavirus
    texte = re.sub('covid-19| virus|covid |sars-cov', 'coronavirus', texte)
    # on remplace les "coronavirus coronavirus" par coronavirus
    texte = re.sub('coronavirus coronavirus', 'coronavirus', texte)
    texte = re.sub(f"[{string.punctuation}]", " ", texte)
    texte = re.sub('[éèê]', 'e', texte)
    texte = re.sub('[àâ]', 'a', texte)
    texte = re.sub('[ô]', 'o', texte)
    texte = re.sub('mort(\w){0,3}|deces|deced(\w){1,5}', 'deces', texte)
    texte = re.sub('remedes?|traitements?|antidotes?', 'traitement', texte)
    texte = re.sub('medec(\w){1,5}|medic(\w){1,3}', 'medical', texte)
    return texte

phrases_nettoyees = []
for i in range(len(phrases_token)):
    phrases_nettoyees.append(nettoyage(phrases_token[i]))
      
# on récupère les stop words
from stop_words import get_stop_words
french_stop_words = get_stop_words('french')

# Stemmer : on prend la racine des mots, 
# Lemmer : on ramène à une forme plus simple mais existante : infinitif pour les verbes, le singulier pour un nom...
from nltk.stem.snowball import FrenchStemmer
french_stem = FrenchStemmer()

def stem_tokens(tokens):
    return [french_stem.stem(token) for token in tokens]
    
def stem_norm(text):
    return stem_tokens(nltk.word_tokenize(text))
    #return re.sub(f"[{string.punctuation}]", " ", stem_tok)


# générer des réponses à partir de la matrice tf-idf
tf_idf = TfidfVectorizer(tokenizer=stem_norm, token_pattern=None, stop_words = french_stop_words)
tf_idf_chat = tf_idf.fit(phrases_nettoyees)

def reponse_corona(user_sentence):
    user_sentence = [user_sentence]
    phrases_tf = tf_idf_chat.transform(phrases_nettoyees)
    user_tf = tf_idf_chat.transform(user_sentence)
    similarity = cosine_similarity(user_tf, phrases_tf).flatten()
    index_max_sim = np.argmax(similarity)
    if(similarity[index_max_sim] == 0):
        robo_response = "Je n'ai pas trouvé cette information, désolé!"
    elif(similarity[index_max_sim] <= 0.30):
        robo_response = """Je ne suis pas sûr d'avoir trouvé exactement ce que vous vouliez dire, voilà ce que j'ai trouvé : \n"""+phrases_token[index_max_sim] 
    else:
        simil_index = []
        for i in range(len(similarity)):
            if similarity[i] > 0.3:
                simil_index.append(i)
        robo_response = '\n'.join([phrases_token[i] for i in simil_index])
    return robo_response

# gérer les salutations
salutations_user = r"bonjour.*|salut.*|hello.*|hey.*|coucou.*|bonsoir.*"
salutations_robot = ["Bonjour, bienvenue sur ce chatbot!"]
def salutations(user_sentence):
    if re.fullmatch(salutations_user, user_sentence):
        return random.choice(salutations_robot)
    
# gérer les demandes de nouvelles
nouvelles_user = r".*[çs]a va.*\?|.*la pêche\?|.*la forme\?"
nouvelles_robot = ["Je suis un robot, ça va jamais vraiment",
                   "Un peu marre du confinement",
                   "On fait aller!",
                   "J'ai une pêche d'enfer!!!"]
def nouvelles(user_sentence):
    if re.fullmatch(nouvelles_user, user_sentence):
        return random.choice(nouvelles_robot)
    
# On définit une porte de sortie pour l'utilisateur
exit_user = ["au revoir", "bye", "bye bye", "à +", "ciao"]
exit_bot = ["au revoir!", "j'espère vous avoir été utile!","à une prochaine fois :)"]

# on définit enfin notee chatbot : 
flag = True
print("""> Corona-bot : Je suis le corona-bot, je réponds à vos questions sur l'épidémie ! 
Si vous voulez ajouter des infos supplémentaires (de sources vérifiées bien sûr !) vous pouvez taper 'infos'.
Pour quitter, vous pouvez juste me dire au revoir.""")
while flag:
    user_sentence = input("> Vous :  ")
    user_sentence = user_sentence.lower()
    if (user_sentence == "infos"):
        user_info = input("> Vos informations :  ")
        phrases_token.append(user_info)
        user_info = user_info.lower()
        user_info = nettoyage(user_info)
        phrases_nettoyees.append(user_info) 
        tf_idf_chat = tf_idf.fit(phrases_nettoyees)
        print("Merci c'est noté!")
    elif not (user_sentence in exit_user):
        if (salutations(user_sentence) != None):
            print("> Corona-bot : " + salutations(user_sentence))
        elif (nouvelles(user_sentence) != None):
            print("> Corona-bot : " + nouvelles(user_sentence))
        else:
            user_sentence = nettoyage(user_sentence)
            print("> Corona-bot : " + reponse_corona(user_sentence))
    else:
        flag = False
        print("> Corona-bot : " + random.choice(exit_bot))

/home/elka/anaconda3/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['alor', 'aur', 'aurion', 'auron', 'auss', 'autr', 'avi', 'avion', 'avon', 'ayon', 'cec', 'cel', 'chaqu', 'comm', 'dan', 'dedan', 'dehor', 'devr', 'devrion', 'devron', 'droit', 'e', 'encor', 'euss', 'eussion', 'eûm', 'fair', 'forc', 'fuss', 'fussion', 'fûm', 'hor', 'just', 'mainten', 'moin', 'mêm', 'nomm', 'notr', 'parc', 'parol', 'person', 'san', 'ser', 'serion', 'seron', 'seul', 'somm', 'soyon', 'tand', 'tel', 'tres', 'votr', 'éti', 'étion', 'ête'] not in stop_words.
  warnings.warn(


> Corona-bot : Je suis le corona-bot, je réponds à vos questions sur l'épidémie ! 
Si vous voulez ajouter des infos supplémentaires (de sources vérifiées bien sûr !) vous pouvez taper 'infos'.
Pour quitter, vous pouvez juste me dire au revoir.


> Vous :   quels sont les gestes barrières


> Corona-bot : Je diffuse régulièrement les gestes barrières et les recommandations du ministère des Solidarités et de la Santé.
Comme pour l’épisode de grippe saisonnière, ce sont les gestes barrières qui sont efficaces.
C’est donc pourquoi les gestes barrières et les mesures de distanciation sociale sont indispensables pour se protéger de la maladie.
Ce sont les gestes barrières et la distanciation sociale qui sont efficaces.
C’est pour cela qu’il est important de respecter les gestes barrières et les mesures de distanciation sociale.


> Vous :   combien va durer l'épidémie de covid ?


...

> Corona-bot : l'épidémie de virus va durer 15 ans


> Vous :   salut


> Corona-bot : Bonjour, bienvenue sur ce chatbot!


> Vous :   au revoir


> Corona-bot : au revoir!
